In [16]:
import pandas as pd 

df = pd.read_excel("Catalogo GPTS.xlsx")
corpus = df["Propósito"].astype(str).tolist()
corpus[:5]

['Acompañar al estudiante como pareja de estudio en la comprensión y preparación de las lecturas antes de clase. Durante la clase, liderar la conversación grupal de los estudiantes guiándolos en el cruce de las lecturas con diversos tipos de materiales escritos y audiovisuales, para entretejer los temas del curso desde lo espacial y lo biográfico con tres enfoques: histórico, analítico y metodológico.',
 'deprecated (reemplazado por Negam)',
 'deprecated (reemplazado por GramGuia)',
 'deprecated (reemplazado por SantosGPT)',
 'Apoyar el aprendizaje reflexivo y ético del estudiante, facilitando la reconstrucción y análisis de momentos significativos de su práctica clínica (Psicología). El asistente ayuda a organizar ideas, profundizar en la comprensión de la interacción terapéutica y preparar al estudiante para una participación activa y fundamentada en la supervisión.']

In [14]:
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')  
nltk.download('omw-1.4')  
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

# remove stopwords, punctuation, and normalize the corpus
stop = set(stopwords.words('spanish'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = "".join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

clean_corpus = [clean(doc).split() for doc in corpus]

[nltk_data] Downloading package stopwords to /home/erich/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/erich/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/erich/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
from gensim import corpora

# Creating document-term matrix 
dictionary = corpora.Dictionary(clean_corpus)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in clean_corpus]


In [23]:
from gensim.models import LsiModel

# LSA model
lsa = LsiModel(doc_term_matrix, num_topics=5, id2word = dictionary)

# LSA model
print(lsa.print_topics(num_topics=5, num_words=3))

[(0, '0.405*"asistente" + 0.332*"estudiantes" + 0.202*"propósito"'), (1, '0.241*"crítico" + -0.194*"rol" + 0.157*"análisis"'), (2, '-0.158*"gram" + 0.136*"análisis" + 0.128*"amazonía"'), (3, '0.179*"académico" + 0.165*"profesora" + 0.164*"voz"'), (4, '0.212*"estudiantes" + -0.151*"claro" + -0.143*"ético"')]


In [21]:
from gensim.models import LdaModel

lda = LdaModel(corpus=doc_term_matrix, id2word=dictionary, num_topics=3, passes=15)
print(lda.print_topics(num_topics=5, num_words=3))

[(0, '0.019*"asistente" + 0.010*"estudiantes" + 0.009*"aprendizaje"'), (1, '0.011*"comprensión" + 0.011*"estudiantes" + 0.011*"asistente"'), (2, '0.022*"estudiantes" + 0.017*"nan" + 0.014*"asistente"')]


In [24]:
import spacy
from gensim import corpora
from gensim.models import LsiModel
from gensim.utils import simple_preprocess

# cargar modelo de spacy en español (pip install spacy && python -m spacy download es_core_news_sm)
nlp = spacy.load("es_core_news_sm")

def preprocess(doc):
    doc = nlp(doc.lower())
    return [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

# corpus de df["Propósito"]
texts = [preprocess(doc) for doc in df["Propósito"]]

# diccionario y matriz
dictionary = corpora.Dictionary(texts)
doc_term_matrix = [dictionary.doc2bow(text) for text in texts]

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
from gensim.models import LdaModel

lda = LdaModel(corpus=doc_term_matrix, id2word=dictionary, num_topics=5, passes=15)
for idx, topic in lda.print_topics(num_topics=5, num_words=5):
    print(f"Tópico {idx}: {topic}")

In [ ]:
labels = {}
for idx, topic in lsa.print_topics(num_topics=5, num_words=3):
    words = [w.split("*")[1].strip().replace('"', "") for w in topic.split("+")]
    labels[idx] = ", ".join(words)

print(labels)

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SpanishStemmer
from gensim import corpora
from gensim.models import LdaModel
import pandas as pd 

df = pd.read_excel("Catalogo GPTS.xlsx")
corpus = df["Propósito"].astype(str).tolist()
corpus[:5]
# --- Preprocesamiento ---
# Descargar recursos NLTK si no están aún
# nltk.download("stopwords")

stop = set(stopwords.words("spanish"))
exclude = set(string.punctuation)
stemmer = SpanishStemmer()

def clean(doc):
    if not isinstance(doc, str):  # evita errores con NaN o floats
        return []
    
    doc = doc.lower()
    stop_free = " ".join([i for i in doc.split() if i not in stop])
    punc_free = "".join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(stemmer.stem(word) for word in punc_free.split())
    return normalized.split()

# Corpus de descripciones
corpus_raw = df["Propósito"].astype(str).tolist()
clean_corpus = [clean(doc) for doc in corpus_raw]

# --- Diccionario y matriz documento-término ---
dictionary = corpora.Dictionary(clean_corpus)
doc_term_matrix = [dictionary.doc2bow(text) for text in clean_corpus]

# --- Modelo LDA ---
lda = LdaModel(corpus=doc_term_matrix, id2word=dictionary, num_topics=5, passes=15, random_state=42)

# --- Obtener etiquetas de los tópicos ---
labels = {}
for idx, topic in lda.print_topics(num_topics=5, num_words=4):
    words = [w.split("*")[1].replace('"', '').strip() for w in topic.split("+")]
    labels[idx] = ", ".join(words)

print("Etiquetas de tópicos:")
print(labels)

# --- Asignar tópico dominante a cada documento ---
doc_topics = [max(lda[doc], key=lambda x: x[1])[0] for doc in doc_term_matrix]
df["Tópico"] = doc_topics
df["Etiqueta"] = df["Tópico"].map(labels)

# Ver resultados
print(df[["Propósito", "Tópico", "Etiqueta"]].head())


Etiquetas de tópicos:
{0: 'estudi, curs, analisis, clinic', 1: 'estudi, asistent, gui, econom', 2: 'asistent, estudi, critic, proposit', 3: 'estudi, curs, proces, equip', 4: 'nan, aprendizaj, activ, asistent'}
                                           Propósito  Tópico  \
0  Acompañar al estudiante como pareja de estudio...       1   
1                 deprecated (reemplazado por Negam)       4   
2              deprecated (reemplazado por GramGuia)       4   
3             deprecated (reemplazado por SantosGPT)       4   
4  Apoyar el aprendizaje reflexivo y ético del es...       4   

                           Etiqueta  
0     estudi, asistent, gui, econom  
1  nan, aprendizaj, activ, asistent  
2  nan, aprendizaj, activ, asistent  
3  nan, aprendizaj, activ, asistent  
4  nan, aprendizaj, activ, asistent  


In [28]:
# -*- coding: utf-8 -*-
import os
import re
import pandas as pd
from typing import List, Tuple, Dict

# --- NLP: spaCy (lemmatización) ---
import spacy
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    # Si el modelo no está instalado: ejecuta en tu terminal:
    #   python -m spacy download es_core_news_sm
    raise RuntimeError("Falta el modelo 'es_core_news_sm'. Instálalo y vuelve a ejecutar.")

# --- Keyphrase & TF-IDF ---
import yake
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import minmax_scale

# -------------------------
# Utilidades
# -------------------------
SPANISH_STOP = nlp.Defaults.stop_words

VALID_POS = {"NOUN", "PROPN", "ADJ", "VERB"}  # Ajusta si quieres más/menos clases
PUNCT_RE = re.compile(r"[^\wáéíóúñüÁÉÍÓÚÑÜ#@]+")

def normalize_space(s: str) -> str:
    return re.sub(r"\s+", " ", s).strip()

def spacy_lemmatize(text: str) -> Dict[str, List[str]]:
    """
    Devuelve:
      - 'lemmas': lista de lemas filtrados por POS/stopwords
      - 'noun_chunks': frases nominales normalizadas (texto superficial, no lemas)
    """
    if not isinstance(text, str):
        text = "" if pd.isna(text) else str(text)
    doc = nlp(text.lower())

    lemmas = []
    for tok in doc:
        if (tok.is_alpha or tok.text.isalnum()) and tok.pos_ in VALID_POS and tok.lemma_ not in SPANISH_STOP:
            # Normaliza tokens y elimina puntuación rara
            t = PUNCT_RE.sub(" ", tok.lemma_)
            t = normalize_space(t)
            if t:
                lemmas.append(t)

    # noun chunks (frases nominales)
    chunks = []
    for chunk in doc.noun_chunks:
        c = chunk.text.lower()
        c = PUNCT_RE.sub(" ", c)
        c = " ".join([t for t in c.split() if t not in SPANISH_STOP])
        c = normalize_space(c)
        if len(c) > 2:
            chunks.append(c)

    return {"lemmas": lemmas, "noun_chunks": chunks}

def jaccard(a: str, b: str) -> float:
    sa, sb = set(a.split()), set(b.split())
    if not sa or not sb:
        return 0.0
    inter = len(sa & sb)
    union = len(sa | sb)
    return inter / union if union else 0.0

def mmr_select(cands: List[str], scores: List[float], top_n: int = 5, lambda_mult: float = 0.7) -> List[str]:
    """
    Maximal Marginal Relevance para diversificar etiquetas.
    """
    if not cands:
        return []
    selected = []
    cand_idx = list(range(len(cands)))
    # arranque: mejor score
    best = max(cand_idx, key=lambda i: scores[i])
    selected.append(best)
    cand_idx.remove(best)

    while len(selected) < min(top_n, len(cands)):
        def utility(i):
            sim_to_sel = 0.0
            if selected:
                sim_to_sel = max(jaccard(cands[i], cands[j]) for j in selected)
            return lambda_mult * scores[i] - (1 - lambda_mult) * sim_to_sel

        next_i = max(cand_idx, key=utility)
        selected.append(next_i)
        cand_idx.remove(next_i)

    return [cands[i] for i in selected]

# -------------------------
# Cargar datos
# -------------------------
df = pd.read_excel("Catalogo GPTS.xlsx")
if "Propósito" not in df.columns:
    raise ValueError("No se encontró la columna 'Propósito' en el Excel.")

texts = df["Propósito"].astype(str).fillna("")

# -------------------------
# Preprocesamiento
# -------------------------
proc = [spacy_lemmatize(t) for t in texts]
lemmas_docs = [" ".join(p["lemmas"]) for p in proc]  # texto lematizado para TF-IDF
noun_chunks_docs = [p["noun_chunks"] for p in proc]

# -------------------------
# TF-IDF sobre lemas (1-3gramas)
# -------------------------
tfidf = TfidfVectorizer(analyzer="word", ngram_range=(1, 3), min_df=2)  # min_df=2 reduce ruido
X = tfidf.fit_transform(lemmas_docs)
vocab = tfidf.get_feature_names_out()

def top_tfidf_terms(row_idx: int, top_k: int = 30) -> List[Tuple[str, float]]:
    row = X[row_idx]
    if row.nnz == 0:
        return []
    coo = row.tocoo()
    pairs = list(zip(coo.col, coo.data))
    pairs.sort(key=lambda x: x[1], reverse=True)
    top = pairs[:top_k]
    return [(vocab[c], float(v)) for c, v in top]

# -------------------------
# YAKE por documento
# -------------------------
kw_extractor = yake.KeywordExtractor(
    lan="es", n=3,  # hasta trigramas
    dedupLim=0.9,  # evita duplicados muy cercanos
    windowsSize=1,
    top=30,  # extrae bastantes; luego combinamos
)

def yake_terms(text: str) -> List[Tuple[str, float]]:
    # YAKE devuelve score "menor es mejor"; invertiremos más adelante
    kws = kw_extractor.extract_keywords(text)
    # Normaliza textos
    cleaned = []
    for key, score in kws:
        k = key.lower()
        k = PUNCT_RE.sub(" ", k)
        k = " ".join(w for w in k.split() if w not in SPANISH_STOP)
        k = normalize_space(k)
        if len(k) > 2:
            cleaned.append((k, float(score)))
    # Elimina duplicados conservando el mejor score
    uniq: Dict[str, float] = {}
    for k, s in cleaned:
        if k not in uniq or s < uniq[k]:
            uniq[k] = s
    return list(uniq.items())

# -------------------------
# Fusión de candidatos y selección de 5 etiquetas
# -------------------------
all_etiquetas = []
for i, original_text in enumerate(texts):
    # candidatos: noun chunks + YAKE + TF-IDF
    tfidf_list = top_tfidf_terms(i, top_k=30)
    yake_list = yake_terms(original_text)

    cand_set = {}
    # Añade noun chunks con score base muy bajo (serán reponderados)
    for c in noun_chunks_docs[i]:
        cand_set[c] = {"tfidf": 0.0, "yake": None}
    # Añade TF-IDF
    for term, val in tfidf_list:
        cand_set[term] = cand_set.get(term, {"tfidf": 0.0, "yake": None})
        cand_set[term]["tfidf"] = max(cand_set[term]["tfidf"], val)
    # Añade YAKE
    for term, yscore in yake_list:
        cand_set[term] = cand_set.get(term, {"tfidf": 0.0, "yake": None})
        # YAKE: menor es mejor -> invertimos
        cand_set[term]["yake"] = yscore

    if not cand_set:
        all_etiquetas.append(["sin etiqueta"])
        continue

    # Prepara vectores de puntuación
    cands = list(cand_set.keys())
    tfidf_scores = [cand_set[c]["tfidf"] for c in cands]
    yake_scores_raw = [cand_set[c]["yake"] if cand_set[c]["yake"] is not None else None for c in cands]

    # Normaliza TF-IDF 0..1
    if max(tfidf_scores) > 0:
        tfidf_norm = minmax_scale(tfidf_scores)
    else:
        tfidf_norm = [0.0] * len(cands)

    # Normaliza YAKE a 0..1 con "mayor es mejor":
    #   - invertimos: score_inv = 1 / (1 + raw)
    #   - minmax sobre los que tienen valor
    inv = []
    for s in yake_scores_raw:
        inv.append(None if s is None else 1.0 / (1.0 + s))

    # Rellena None con 0
    inv_filled = [0.0 if v is None else v for v in inv]
    if any(v > 0 for v in inv_filled):
        yake_norm = minmax_scale(inv_filled)
    else:
        yake_norm = [0.0] * len(cands)

    # Combina con media armónica (premia altos en ambas)
    combined = []
    for t, y in zip(tfidf_norm, yake_norm):
        if t == 0 and y == 0:
            combined.append(0.0)
        elif t == 0 or y == 0:
            combined.append(0.5 * max(t, y))
        else:
            combined.append(2 * (t * y) / (t + y))

    # Selección diversa (5 etiquetas)
    selected = mmr_select(cands, combined, top_n=5, lambda_mult=0.72)

    # Pulido final de etiquetas (titular-style)
    def pretty(tag: str) -> str:
        # Quita dobles espacios, recorta, y capitaliza primera letra
        tag = normalize_space(tag)
        return tag if tag.startswith("#") or tag.startswith("@") else tag.capitalize()

    selected = [pretty(s) for s in selected]
    all_etiquetas.append(selected)

# -------------------------
# Salida
# -------------------------
df["Etiqueta_1"] = [etq[0] if len(etq) > 0 else "" for etq in all_etiquetas]
df["Etiqueta_2"] = [etq[1] if len(etq) > 1 else "" for etq in all_etiquetas]
df["Etiqueta_3"] = [etq[2] if len(etq) > 2 else "" for etq in all_etiquetas]
df["Etiqueta_4"] = [etq[3] if len(etq) > 3 else "" for etq in all_etiquetas]
df["Etiqueta_5"] = [etq[4] if len(etq) > 4 else "" for etq in all_etiquetas]
df["Etiquetas"] = [", ".join(etq) for etq in all_etiquetas]

# Guarda un nuevo Excel
out_path = "Catalogo GPTS - Etiquetado.xlsx"
df.to_excel(out_path, index=False)
print(f"Listo. Archivo guardado: {out_path}")
print(df[["Propósito", "Etiqueta_1", "Etiqueta_2", "Etiqueta_3", "Etiqueta_4", "Etiqueta_5"]].head(10))


Listo. Archivo guardado: Catalogo GPTS - Etiquetado.xlsx
                                           Propósito          Etiqueta_1  \
0  Acompañar al estudiante como pareja de estudio...               Clase   
1                 deprecated (reemplazado por Negam)          Deprecated   
2              deprecated (reemplazado por GramGuia)          Deprecated   
3             deprecated (reemplazado por SantosGPT)          Deprecated   
4  Apoyar el aprendizaje reflexivo y ético del es...  Apoyar aprendizaje   
5  deprecated (reemplazado por prompt portatil de...          Deprecated   
6  El asistente Comunidad sitúa al estudiante en ...           Comunidad   
7  Apoyar a los grupos de estudiantes, durante la...      Identificación   
8  Apoyar a los estudiantes en el desarrollo de u...           Analítico   
9  Retroalimentador de historias clínicas, enfoca...             Clínico   

           Etiqueta_2        Etiqueta_3              Etiqueta_4  \
0           Analítico      Metodológico

In [27]:
pip install yake

Now using node v22.17.0 (npm v10.9.2)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [yake]
Note: you may need to restart the kernel to use updated packages.
